In [9]:
import os
import numpy as np
from core.data_provider import datasets_factory
from core.models.model_factory import Model
from core.utils import preprocess
import core.trainer as trainer
import math 
# export CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [10]:
class configs:
    is_training = 0
    device = "cuda"
    dataset_name = "motion"
    valid_data_paths = "/home/xiao/Projects/ST_LSTM_MP/Data/2D_map_64/Map_array/normalized_test_seen_video.npz"
    gen_frm_dir = "results/motion_predrnn"
    input_length = 1
    total_length = 12
    img_width = 64
    img_channel = 3
    model_name = "predrnn"# "predrnn_memory_decoupling"
    pretrained_model = "checkpoints/model.ckpt-57000"
    num_hidden = "128,128,128,128"
    filter_size = 5
    stride = 1
    patch_size = 4 
    batch_size = 1
    layer_norm = 0
    lr = 0.0003
    reverse_scheduled_sampling = 0
    num_save_samples = 10
    decouple_beta = 0.1
    visual = False
    visual_path = ""

In [11]:
import matplotlib.pyplot as plt
# test_seen_video = np.load("/home/xiao/Projects/ST_LSTM_MP/Data/2D_map_64/Map_array/test_seen_video.npy")
# test_seen_video = np.transpose(test_seen_video, (0,1,2,4,5,3))
# print ('test_seen_video', test_seen_video.shape)
# test_seen_config = np.load("/home/xiao/Projects/ST_LSTM_MP/Data/2D_map_64/Map_array/test_seen_config.npy")
# video_len = test_seen_video.shape[2]
# frame_size = test_seen_video.shape[3]
# frame_channel = test_seen_video.shape[-1]
# print ('test_seen_config', test_seen_config.shape)

test_seen_video (100, 200, 12, 64, 64, 3)
test_seen_config (100, 400, 4)


In [18]:
# small_test_env = test_seen_video[:, :3, 0, :, :, :]
# small_test_config = test_seen_config[:, :3, :]
# np.save("Data/rf_64_test.npy", small_test_env)
# np.save("Data/rf_config_test.npy", small_test_config)

In [20]:
test_env = np.load("Data/rf_64_test.npy")
test_config = np.load("Data/rf_config_test.npy")
print (test_env.shape)

(100, 3, 64, 64, 3)


In [21]:
model = Model(configs)
model.load(configs.pretrained_model)

load model: checkpoints/model.ckpt-57000


In [22]:
def augmentation(img, start_x, start_y):
#     start_x, start_y = np.where(img[1]==255)
    h, w = 7, 7
#     for i range(-3, 4):
#         for j in range(-3, 4):
#             x, y = start_x + i, start_y + j
#             if img[0,x,y] != 255:
#                 img[1,x,y] = 255
#     start_x, start_y = np.where(img[2]==255)
    for i in range(-3, 4):
        for j in range(-3, 4):
            x, y = start_x + i, start_y + j
            if 0<=x<=(img.shape[0]-1) and 0 <=y<=(img.shape[0]-1) and img[x,y,0]<0.1:
                img[x,y, 1] = 1.0
    return img

In [23]:
def IsInCollision(curr_x, curr_y, obs):
    cf = False
    for i in range(obs.shape[0]):
#         print ('checking obstacle', i)
        if abs(obs[i, 0] - curr_x) < obs[i,2]/2.0 and  abs(obs[i, 1] - curr_y) < obs[i,3]/2.0:
            return True
    return False

In [27]:
import time

Threshold = 0.25
DISCRETIZATION_STEP= 0.01
max_itr = 20
max_sample = 10
success_rate = 0
all_path = []
all_time = 0
valid_task = []


for map_id in range(test_env.shape[0]):
    print ('Testing seen map', map_id)
    map_path = []
    obs = all_obs[map_id]

    for task_id in range(test_env.shape[1]):
        start_x, start_y = test_config[map_id, task_id, :-2]
        goal_x, goal_y = test_config[map_id, task_id, -2:]
        path = [(start_x, start_y)]
        print ('configuration', test_config[map_id, task_id])
        
        first_frame = np.copy(test_env[map_id,task_id])
        video = np.ones((5,configs.total_length,frame_size,frame_size,frame_channel)) / 255.0
        video[0, :, :, :, 0] = first_frame[:,:,0]
        video[0, :, :, :, 2] = first_frame[:,:,2]
        video[0, 0, :, :, 1] = first_frame[:,:,1] #the intial state
        configs.input_length = 1
        real_input_flag = np.zeros(
                (configs.batch_size,
                 configs.total_length - configs.input_length - 1,
                 configs.img_width // configs.patch_size,
                 configs.img_width // configs.patch_size,
                 configs.patch_size ** 2 * configs.img_channel))
        
        no_step = 0
        no_frame = 0
        s_time = time.time()
        while True:
            if no_step >= configs.total_length - 2:
                no_frame = configs.total_length - 2
            else:
                no_frame = no_step

            test_dat = preprocess.reshape_patch(video, configs.patch_size)          
            predicted_frame = model.test(test_dat, real_input_flag)        
            predicted_frame = preprocess.reshape_patch_back(predicted_frame, configs.patch_size)
  
            predicted_pos = predicted_frame[0,no_frame,:,:,1]
            sample_pools = np.unravel_index(np.argsort(predicted_pos, axis=None)[::-1], predicted_pos.shape)
            
            for idx in range(sample_pools[0].shape[0]):
                next_x, next_y = sample_pools[0][idx], sample_pools[1][idx]
                if first_frame[next_x, next_y, 0] > 0.9 or (next_x, next_y) in path:
                    continue
                last_dir = np.sign([path[-1][0] - start_x, path[-1][1] - start_y])
                curr_dir = np.sign([next_x - path[-1][0], next_y - path[-1][1]])
                if np.sum(np.abs(curr_dir - last_dir)) >= 4:
                    continue
                    
                #collision detection
                last_x, last_y = path[-1]
                distTotal = math.sqrt((next_x - last_x)**2 + (next_y - last_y)**2)
                
                collision_flag = False
                if distTotal>0:
                    incrementTotal = distTotal/DISCRETIZATION_STEP
                    numSegments = int(math.floor(incrementTotal))
                    x_dist = (next_x - last_x)/numSegments
                    y_dist = (next_y - last_y)/numSegments
                                     
                    for pt in range(numSegments):
                        last_x = last_x + x_dist
                        last_y = last_y + y_dist
                        if first_frame[int(last_x), int(last_y), 0] > 0.9:
                            collision_flag = True
                            break
                        
                if not collision_flag:
                    no_step += 1
                    path.append((next_x, next_y))
                    break
                    
            if abs(next_x - goal_x) + abs(next_y - goal_y) <= 6:
                success_rate += 1
                path.append((goal_x, goal_y))
                path.append('#') #denote the path is successful
                #             ############
                if (len(path) < 8):
                    valid_task.append((map_id, task_id))
                break
                
            if no_step >= configs.total_length - 2:
                video[0, :-1] = video[0, 1:]
                video[0, -2] = augmentation(video[0, -2], next_x, next_y)
            else:
                video[0, no_frame + 1] = augmentation(video[0, no_frame + 1], next_x, next_y)
            configs.input_length = no_frame + 1
            real_input_flag = np.zeros(
                    (configs.batch_size,
                     configs.total_length - configs.input_length - 1,
                     configs.img_width // configs.patch_size,
                     configs.img_width // configs.patch_size,
                     configs.patch_size ** 2 * configs.img_channel))
            
            if no_step > max_itr:
                print ('failed!')
                break
        all_time += time.time() - s_time
        print ('path', path)
        map_path.append(path)
    all_path.append(map_path)
    print ('sucess_rate', success_rate)
    
print ('average length', sum([len(p) for p in all_path])/len(all_path))
print ('The average time is', all_time/8000)        

Testing seen map 0
configuration [ 1 60 58 13]
path [(1, 60), (8, 28), (20, 15), (52, 3), (62, 18), (57, 16), (58, 13), '#']
configuration [59  2  0 56]
path [(59, 2), (46, 8), (16, 19), (13, 20), (8, 46), (0, 57), (0, 56), '#']
configuration [ 2 13 62 53]
path [(2, 13), (21, 18), (39, 27), (39, 47), (59, 56), (62, 53), '#']
sucess_rate 3
Testing seen map 1
configuration [60 62 18  0]
path [(60, 62), (51, 50), (46, 35), (53, 31), (41, 15), (17, 0), (18, 0), '#']
configuration [13  8 60 61]
path [(13, 8), (42, 19), (49, 25), (60, 39), (60, 60), (60, 61), '#']
configuration [ 0  4 42 62]
path [(0, 4), (20, 7), (49, 26), (52, 25), (43, 52), (43, 61), (42, 62), '#']
sucess_rate 6
Testing seen map 2
configuration [59 51  0  3]
path [(59, 51), (35, 23), (22, 12), (0, 1), (0, 3), '#']
configuration [54 61  1  5]
path [(54, 61), (52, 52), (44, 36), (26, 23), (19, 31), (15, 17), (0, 4), (1, 5), '#']
configuration [ 1 56 56 10]
path [(1, 56), (4, 51), (18, 44), (42, 15), (55, 12), (56, 10), '#']

path [(56, 4), (34, 9), (28, 18), (8, 57), (6, 58), '#']
configuration [ 8 55 60  1]
path [(8, 55), (13, 45), (22, 45), (47, 42), (43, 39), (55, 43), (35, 39), (46, 10), (45, 10), (60, 1), (60, 1), '#']
configuration [ 6 58 57  3]
path [(6, 58), (13, 46), (29, 46), (45, 24), (51, 43), (54, 6), (57, 3), '#']
sucess_rate 75
Testing seen map 25
configuration [61  2  6 53]
path [(61, 2), (46, 8), (19, 19), (11, 26), (20, 33), (11, 55), (8, 55), (6, 53), '#']
configuration [56 10  2 56]
path [(56, 10), (44, 14), (21, 20), (16, 30), (11, 58), (11, 55), (0, 57), (2, 56), '#']
configuration [ 0 53 60 12]
path [(0, 53), (3, 48), (25, 37), (43, 36), (53, 33), (56, 17), (57, 9), (60, 12), '#']
sucess_rate 78
Testing seen map 26
configuration [57 58 10  1]
path [(57, 58), (51, 57), (48, 49), (42, 47), (28, 35), (15, 27), (2, 18), (2, 3), (13, 3), (10, 1), '#']
configuration [59 54  4  5]
path [(59, 54), (54, 49), (44, 43), (28, 32), (11, 29), (7, 21), (4, 19), (4, 3), (4, 5), '#']
configuration [6

path [(61, 2), (33, 16), (25, 26), (1, 46), (1, 44), '#']
configuration [17  4 60 61]
path [(17, 4), (20, 27), (49, 25), (62, 57), (60, 61), '#']
sucess_rate 147
Testing seen map 49
configuration [61  9  0 48]
path [(61, 9), (39, 28), (25, 36), (12, 46), (0, 50), (0, 48), '#']
configuration [58 58  6  2]
path [(58, 58), (40, 43), (21, 35), (12, 23), (3, 5), (6, 2), '#']
configuration [ 3  1 54 56]
path [(3, 1), (8, 18), (20, 35), (31, 38), (46, 49), (54, 57), (54, 56), '#']
sucess_rate 150
Testing seen map 50
configuration [ 7  1 56 60]
path [(7, 1), (22, 15), (40, 39), (54, 60), (56, 60), '#']
configuration [ 6  6 55 60]
path [(6, 6), (18, 11), (54, 63), (55, 60), '#']
configuration [58  4  3 57]
path [(58, 4), (35, 23), (22, 32), (17, 38), (13, 45), (6, 56), (3, 57), '#']
sucess_rate 153
Testing seen map 51
configuration [60  3  2 51]
path [(60, 3), (57, 18), (42, 27), (24, 39), (0, 53), (2, 51), '#']
configuration [ 0  9 62 62]
path [(0, 9), (8, 13), (25, 19), (35, 25), (45, 44), (6

path [(1, 52), (19, 29), (24, 24), (11, 13), (56, 9), (57, 8), '#']
configuration [62  0  0 56]
path [(62, 0), (48, 6), (27, 16), (0, 57), (0, 56), '#']
configuration [60 55  1  7]
path [(60, 55), (34, 41), (2, 17), (32, 17), (0, 5), (1, 7), '#']
sucess_rate 219
Testing seen map 73
configuration [ 5 12 61 61]
path [(5, 12), (19, 19), (39, 26), (45, 30), (60, 61), (61, 61), '#']
configuration [57 61  4 14]
path [(57, 61), (24, 46), (17, 24), (7, 16), (4, 14), '#']
configuration [57 59  0 10]
path [(57, 59), (24, 45), (23, 40), (9, 9), (3, 13), (0, 10), '#']
sucess_rate 222
Testing seen map 74
configuration [ 2 12 53 61]
path [(2, 12), (22, 19), (45, 39), (56, 60), (53, 61), '#']
configuration [57 50  6  1]
path [(57, 50), (48, 40), (40, 39), (7, 3), (6, 1), '#']
configuration [ 5 61 60  7]
path [(5, 61), (20, 43), (33, 31), (40, 15), (57, 17), (57, 8), (60, 7), '#']
sucess_rate 225
Testing seen map 75
configuration [49  3  2 60]
path [(49, 3), (38, 7), (24, 23), (4, 60), (2, 60), '#']
c

path [(9, 5), (16, 18), (22, 31), (32, 52), (56, 57), (58, 58), '#']
sucess_rate 291
Testing seen map 97
configuration [61 51 10  0]
path [(61, 51), (60, 43), (46, 32), (10, 1), (10, 0), '#']
configuration [ 4  6 61 61]
path [(4, 6), (28, 16), (44, 24), (60, 41), (59, 61), (61, 61), '#']
configuration [ 4 46 62  0]
path [(4, 46), (28, 31), (36, 19), (45, 10), (60, 1), (62, 0), '#']
sucess_rate 294
Testing seen map 98
configuration [59 51  1  7]
path [(59, 51), (56, 28), (39, 23), (21, 20), (8, 14), (2, 10), (1, 7), '#']
configuration [ 0  4 45 62]
path [(0, 4), (30, 21), (40, 19), (56, 31), (55, 36), (45, 61), (45, 62), '#']
configuration [ 7 58 59  6]
path [(7, 58), (11, 51), (25, 48), (38, 36), (49, 23), (55, 22), (63, 11), (59, 8), (59, 6), '#']
sucess_rate 297
Testing seen map 99
configuration [55 57  7  5]
path [(55, 57), (46, 36), (18, 10), (7, 7), (7, 5), '#']
configuration [ 1 59 57  5]
path [(1, 59), (7, 43), (16, 40), (33, 30), (58, 7), (57, 5), '#']
configuration [60 60  2  